In [26]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

读入文本

In [3]:
raw_text = open('schooltech.txt').read()
raw_text = raw_text.lower()

In [4]:
raw_text[:200]

'the project gutenberg ebook, what the schools teach and might teach, by\njohn franklin bobbitt\n\n\nthis ebook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  you'

In [5]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c,i) for i,c in enumerate(chars))
int_to_char = dict((i,c) for i,c in enumerate(chars))

In [6]:
len(raw_text)

140437

In [7]:
type(raw_text)

str

In [8]:
seq_length = 100
x = []
y = []
for i in range(0, len(raw_text)-seq_length):
    in_chars = raw_text[i:i+seq_length]
    out_char = raw_text[i+seq_length]
    x.append([char_to_int[c] for c in in_chars])
    y.append([char_to_int[out_char]])

In [9]:
print(x[:3])

[[53, 41, 38, 1, 49, 51, 48, 43, 38, 36, 53, 1, 40, 54, 53, 38, 47, 35, 38, 51, 40, 1, 38, 35, 48, 48, 44, 12, 1, 56, 41, 34, 53, 1, 53, 41, 38, 1, 52, 36, 41, 48, 48, 45, 52, 1, 53, 38, 34, 36, 41, 1, 34, 47, 37, 1, 46, 42, 40, 41, 53, 1, 53, 38, 34, 36, 41, 12, 1, 35, 58, 0, 43, 48, 41, 47, 1, 39, 51, 34, 47, 44, 45, 42, 47, 1, 35, 48, 35, 35, 42, 53, 53, 0, 0, 0, 53, 41, 42, 52], [41, 38, 1, 49, 51, 48, 43, 38, 36, 53, 1, 40, 54, 53, 38, 47, 35, 38, 51, 40, 1, 38, 35, 48, 48, 44, 12, 1, 56, 41, 34, 53, 1, 53, 41, 38, 1, 52, 36, 41, 48, 48, 45, 52, 1, 53, 38, 34, 36, 41, 1, 34, 47, 37, 1, 46, 42, 40, 41, 53, 1, 53, 38, 34, 36, 41, 12, 1, 35, 58, 0, 43, 48, 41, 47, 1, 39, 51, 34, 47, 44, 45, 42, 47, 1, 35, 48, 35, 35, 42, 53, 53, 0, 0, 0, 53, 41, 42, 52, 1], [38, 1, 49, 51, 48, 43, 38, 36, 53, 1, 40, 54, 53, 38, 47, 35, 38, 51, 40, 1, 38, 35, 48, 48, 44, 12, 1, 56, 41, 34, 53, 1, 53, 41, 38, 1, 52, 36, 41, 48, 48, 45, 52, 1, 53, 38, 34, 36, 41, 1, 34, 47, 37, 1, 46, 42, 40, 41, 53, 1,

In [10]:
print(y[:3])

[[1], [38], [35]]


In [11]:
n_patterns = len(x)
n_vocab = len(chars)

x = np.reshape(x, (n_patterns, seq_length, 1))
x = x/float(n_vocab)
y = np_utils.to_categorical(y)

In [12]:
x.shape

(140337, 100, 1)

In [25]:
type(x)

numpy.ndarray

In [13]:
y.shape

(140337, 61)

构造模型

In [14]:
model = Sequential()
model.add(LSTM(units=128, input_shape=(x.shape[1], x.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer = 'adam')

In [17]:
model.fit(x, y, epochs=1, batch_size=32)

Epoch 1/1
  1184/140337 [..............................] - ETA: 1175s - loss: 2.7523

KeyboardInterrupt: 

In [27]:
def predict_next(input_array):
    x = np.reshape(input_array, (1, seq_length, 1))
    x = x/float(n_vocab)
    y = model.predict(x)
    return y

def string_to_index(raw_input):
    res=[]
    for c in raw_input[(len(raw_input) - seq_length):]:
        res.append(char_to_int[c])
    return res

def y_to_char(y): 
    return int_to_char[y.argmax()]

In [28]:
def generate_article(init, num=200):
    in_string = init.lower()
    for i in range(num):
        n = y_to_char(predict_next(string_to_index(in_string)))
        in_string += n
    return in_string


In [30]:
init = 'Pupils in fact receive only an introductory training in vocal music. The whole field of instrumental'

out = generate_article(init)

print(out)

pupils in fact receive only an introductory training in vocal music. the whole field of instrumental  the tee the tee tee the tee tee tee the tee tee tee the tee tee tee the tee tee tee the tee tee tee the tee tee tee the tee tee tee the tee tee tee the tee tee tee the tee tee tee the tee tee tee th


词word